In [1]:
import pandas as pd
import pickle

# Load learned models from pickle

In [29]:
with open('pickles/predict_rental_price.pkl', 'rb') as f:
    regressor_rental = pickle.load(f) 
with open('pickles/predict_rental_price_scaler.pkl', 'rb') as f:
    scaler_rental = pickle.load(f) 
    
with open('pickles/predict_sale_price.pkl', 'rb') as f:
    regressor_sale = pickle.load(f) 
with open('pickles/predict_sale_price_scaler.pkl', 'rb') as f:
    scaler_sale = pickle.load(f) 

# Build functions capable to use trained model

In [30]:
def predict_rental(test):
    
    test_ = scaler_rental.transform(test)
    display(test)

    print("For an apartment with area = {} m^2, {} room(s), build in {} and located at ({}, {}), \npredicted rental price is {} zł.".
          format(test['Powierzchnia'].iloc[0], test['Liczba pokoi'].iloc[0], test['Rok budowy'].iloc[0]
                 , test['φ'].iloc[0], test['λ'].iloc[0], round(regressor_rental.predict(test_)[0])))
    
    
def predict_sale(test):

    test_ = scaler_sale.transform(test)
    display(test)

    print("For an apartment with area = {} m^2, {} room(s), build in {} and located at ({}, {}), \npredicted sale price is {} zł.".
          format(test['Powierzchnia'].iloc[0], test['Liczba pokoi'].iloc[0], test['Rok budowy'].iloc[0]
                 , test['φ'].iloc[0], test['λ'].iloc[0], round(regressor_sale.predict(test_)[0])))

# Test on some hand-typed data

In [31]:
test_1 = pd.DataFrame([[45, 2, 52.197751, 20.964537, 2018, 25]], columns=['Powierzchnia', 'Liczba pokoi', 'φ', 'λ', 'Rok budowy', 'Czas dojazdu do centrum'])
test_2 = pd.DataFrame([[68.80, 3, 52.147, 21.047, 1980, 29]], columns=['Powierzchnia', 'Liczba pokoi', 'φ', 'λ', 'Rok budowy', 'Czas dojazdu do centrum'])

In [32]:
predict_rental(test_1)
#predict_sale(test_1)

,Powierzchnia,Liczba pokoi,φ,λ,Rok budowy,Czas dojazdu do centrum
0,45,2,52.197751,20.964537,2018,25


For an apartment with area = 45 m^2, 2 room(s), build in 2018 and located at (52.197751, 20.964537), 
predicted rental price is 3052.0 zł.


In [33]:
#predict_rental(test_2)
predict_sale(test_2)

,Powierzchnia,Liczba pokoi,φ,λ,Rok budowy,Czas dojazdu do centrum
0,68.8,3,52.147,21.047,1980,29


For an apartment with area = 68.8 m^2, 3 room(s), build in 1980 and located at (52.147, 21.047), 
predicted sale price is 654283.0 zł.
